In [1]:
import sys
sys.path.append('..\..\src') # this ensures that the src models are located to import them bellow
#sys.path.append('C:\\Users\\Marios\\Desktop\\Μάριος\\BotBattleOfSea\\src') deprecated absolute path

In [2]:
import cv2
import numpy as np
import pyautogui  # Import pyautogui for screen capture
from ultralytics import YOLO
from ultralytics.utils.plotting import colors, Annotator
from PIL import ImageGrab
from win32api import GetSystemMetrics
import keyboard #pip install keyboard
import time
from Paths import *
import math

We will hardwrite all Tools function because we cant use from docs the function getText

In [3]:
time_of_pause=0


In [4]:
def click(middle_x,middle_y):
    pyautogui.click(middle_x, middle_y)
    print("function: click") # for debugging only
    time.sleep(time_of_pause) # for debugging only


def screenshot(x1, y1, x2, y2,i):
    # Convert coordinates to integers
    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

    # Take a screenshot of the specified region
    screenshot = pyautogui.screenshot(region=(x1,y1,x2-x1,y2-y1))

    # Save the screenshot
    screenshot.save(f'OcrTexts\screenshot{i}.png')
    print("function: screenshot") # for debugging only
    time.sleep(time_of_pause) # for debugging only

def screenshot_array(x1, y1, x2, y2,i):
    # Convert coordinates to integers
    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
    
    # Expand the region by 5 pixels
    x1 -= 5
    y1 -= 5
    x2 += 5
    y2 += 5

    # Take a screenshot of the expanded region
    screenshot = pyautogui.screenshot(region=(x1, y1, x2 - x1, y2 - y1))
    screenshot.save(f'C:\\Users\\Marios\\Desktop\\Μάριος\\BotBattleOfSea\\src\\OcrTexts\\OCRtext{i}.png')

    print("function: screenshot_array")  # for debugging only
    time.sleep(time_of_pause)  # for debugging only
    return np.array(screenshot)


#this absolute path works only for ipynb and not for the actual Tools.py file
def getText(screenshot_array_format,modelOCR=YOLO(absolute_path_ocr_model_640px_windows)):
    results = modelOCR.predict(screenshot_array_format,imgsz=640, conf=0.2)
    boxes = results[0].boxes.xyxy.cpu()
    clss = results[0].boxes.cls.cpu().tolist()
    Dictionary = results[0].names
    mapping = {}
    for box, cls in zip(boxes, clss):
        mapping[box[0]] = Dictionary[cls][-1]
        #print(f"box={box[0]}, cls={cls}->{Dictionary[cls]}")
    
    sorted_dict = {k: v for k, v in sorted(mapping.items(), key=lambda item: item[0])}
    string=''
    for value in sorted_dict.values():
        string=string+value

    print("function: getText") # for debugging only
    time.sleep(time_of_pause) # for debugging only
    return string

def mouse_in_safe_zone():
    pyautogui.FAILSAFE = False  # Disables the fail-safe feature
    pyautogui.moveTo(0, 0, duration=0)
    print("function: mouse_in_safe_zone") # for debugging only
    time.sleep(time_of_pause) # for debugging only

import random

def getWidth_Height(x1, x2, y1, y2):
    # Ensure x1 < x2 and y1 < y2
    x_min, x_max = min(x1, x2), max(x1, x2)
    y_min, y_max = min(y1, y2), max(y1, y2)
    
    Width=x_max-x_min
    Height=y_max-y_min

    print("function: getWidth_Height") # for debugging only
    time.sleep(time_of_pause) # for debugging only
    return Width, Height

def getMiddle_point(x1, x2, y1, y2):
    middle_x = (x1 + x2) / 2
    middle_y = (y1 + y2) / 2
    
    print("function: getMiddle_point") # for debugging only
    time.sleep(time_of_pause) # for debugging only
    return(middle_x, middle_y)

def stableCam():
    pyautogui.typewrite('ws')
    print("function: stableCam") # for debugging only
    time.sleep(time_of_pause) # for debugging only

def centerCamera(): # g is default letter for centering the cammera 
    pyautogui.typewrite('g')
    print("function: centerCamera") # for debugging only
    time.sleep(time_of_pause) # for debugging only

def cartesian_distance(x1, y1, x2, y2):
    """
    Calculate the Cartesian distance between two points (x1, y1) and (x2, y2).

    Args:
    - x1, y1 (float): Coordinates of the first point.
    - x2, y2 (float): Coordinates of the second point.

    Returns:
    - float: The Cartesian distance between the two points.
    """
    print(np.array([x2, y2]).shape, np.array([x1, y1]).shape)
    distance = np.linalg.norm(np.array([x2, y2]) - np.array([x1, y1]))
    return distance

def handleOCRtext(objects, final_array, ocr_counter):
    label_17_OCRText = objects[objects[:, 5] == 17] # 17 -> OCRText
    if len(label_17_OCRText) > 0:
        ocr_screenshot = screenshot_array(label_17_OCRText[0][0], label_17_OCRText[0][1], label_17_OCRText[0][2], label_17_OCRText[0][3], ocr_counter)
        Numbers = getText(ocr_screenshot)
        if len(Numbers) == 4:
            label_7_OCRText = final_array[final_array[:, 3] == 7] # 7: 'TextFieldToBotQuestion'
            label_4_OCRText = final_array[final_array[:, 3] == 4] # 4: 'TextFieldToBotQuestion'
            if(len(label_7_OCRText) > 0 and len(label_4_OCRText) > 0):
                click(label_7_OCRText[0][0], label_7_OCRText[0][1])
                time.sleep(random.randint(1, 3))
                pyautogui.typewrite(Numbers)
                time.sleep(random.randint(1, 3))
                pyautogui.click(label_4_OCRText[0][0], label_4_OCRText[0][1])
                return True
    return False


This function bellow is about only to check the results of the OCR detection


In [5]:
import os

def load_images_from_folder(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.jpg') or filename.endswith('.png'): # Adjust file extensions as needed
            img_path = os.path.join(folder_path, filename)
            img = cv2.imread(img_path)
            if img is not None:
                images.append(img)
    return images

folder_path = r'C:\Users\Marios\Desktop\Μάριος\BotBattleOfSea\src\OcrTexts'
folder_path_labeled=r'C:\Users\Marios\Desktop\Μάριος\BotBattleOfSea\docs\Snapshot Storage\OCR\Labeled'
images = load_images_from_folder(folder_path_labeled)

# Now images is a list containing all the loaded images from the folder


In [ ]:
for image in images: 
    print(getText(image))

Lets get Start with the Object Detection System

In [27]:
#This is for checking how the final results should be displayed
!yolo task=detect mode=predict model="C:\\Users\\Marios\\Desktop\\Μάριος\\BotBattleOfSea\\src\\models\\yolov8_2000px.pt" conf=0.25 source='C:\\Users\\Marios\\Desktop\\Μάριος\\BotBattleOfSea\\src\\ObjectDetections' save=True save_txt=True

Ultralytics YOLOv8.1.14 🚀 Python-3.11.2 torch-2.2.0+cpu CPU (AMD Ryzen 5 2600X Six-Core Processor)
Model summary (fused): 168 layers, 3008183 parameters, 0 gradients, 8.1 GFLOPs

image 1/4 C:\Users\Marios\Desktop\Μάριος\BotBattleOfSea\src\ObjectDetections\screenshot1.png: 1152x2016 1 BattleOfSeaChangeSize, 1 BattleOfSeaExit, 1 BattleOfSeaWindow, 1 CenterMyBoat, 1 chatButton, 1 glitter, 1 map, 575.9ms
image 2/4 C:\Users\Marios\Desktop\Μάριος\BotBattleOfSea\src\ObjectDetections\screenshot2.png: 1152x2016 1 BattleOfSeaExit, 1 BattleOfSeaWindow, 1 CenterMyBoat, 1 chatButton, 2 glitters, 1 glitterOnMap, 1 map, 592.7ms
image 3/4 C:\Users\Marios\Desktop\Μάριος\BotBattleOfSea\src\ObjectDetections\screenshot3.png: 1152x2016 1 BattleOfSeaChangeSize, 1 BattleOfSeaExit, 1 BattleOfSeaWindow, 1 CenterMyBoat, 1 chatButton, 1 glitter, 1 glitterOnMap, 1 map, 515.3ms
image 4/4 C:\Users\Marios\Desktop\Μάριος\BotBattleOfSea\src\ObjectDetections\screenshot4.png: 1152x2016 1 BattleOfSeaChangeSize, 1 BattleO

This is default way to make a display by using Yolo fucntions

In [21]:
from ultralytics.models.yolo.detect import DetectionPredictor

model_path = "C:\\Users\\Marios\\Desktop\\Μάριος\\BotBattleOfSea\\src\\models\\yolov8_2000px.pt"
source_path = "C:\\Users\\Marios\\Desktop\\Μάριος\\BotBattleOfSea\\src\\ObjectDetections"
output_dir = "C:\\Users\\Marios\\Desktop\\Μάριος\\BotBattleOfSea\\src\\Output"

args = dict(model=model_path, source=source_path ,imgsz=2000, save=True, save_txt=True, show_boxes=True) # xywh)
predictor = DetectionPredictor(overrides=args)
predictor.predict_cli()






Ultralytics YOLOv8.1.14 🚀 Python-3.11.2 torch-2.2.0+cpu CPU (AMD Ryzen 5 2600X Six-Core Processor)
Model summary (fused): 168 layers, 3009353 parameters, 0 gradients, 8.1 GFLOPs
WARNING ⚠️ imgsz=[2000] must be multiple of max stride 32, updating to [2016]
image 1/2 C:\Users\Marios\Desktop\Μάριος\BotBattleOfSea\src\ObjectDetections\MyBoat34.png: 1152x2016 1 BattleOfSeaChangeSize, 1 BattleOfSeaExit, 3 BattleOfSeaWindows, 1 EnemyClicked, 1 MyShip, 1 chatButton, 1 fireDisabled, 1 map, 1 pointOfMoving, 581.9ms
image 2/2 C:\Users\Marios\Desktop\Μάριος\BotBattleOfSea\src\ObjectDetections\image0.png: 1152x2016 1 BattleOfSeaChangeSize, 1 BattleOfSeaExit, 3 BattleOfSeaWindows, 1 CenterMyBoat, 1 MyShip, 1 chatButton, 1 fireDisabled, 2 glitters, 1 glitterClicked, 1 map, 567.0ms
Speed: 13.7ms preprocess, 574.5ms inference, 0.0ms postprocess per image at shape (1, 3, 1152, 2016)
Results saved to runs\detect\train
2 labels saved to runs\detect\train\labels


We will create a function that will predict

In [6]:

def prepare_model(imgsize):
    global model
    model = YOLO(absolute_path_object_model_2000px_windows)
    folder_path = r'C:\Users\Marios\Desktop\Μάριος\BotBattleOfSea\src\ObjectDetections'
    folder_unlabeled = r'C:\Users\Marios\Desktop\Μάριος\BotBattleOfSea\docs\Snapshot Storage\PlayForGlitter\Unlabeled'
    global images_uint8, results , mapped_list
    # Load images from folder
    images = load_images_from_folder(folder_path)

    images_rgb = [cv2.cvtColor(img, cv2.COLOR_BGR2RGB) for img in images]
    images_uint8 = [img.astype('uint8') for img in images_rgb]

    results = model.predict(images_uint8, imgsz=imgsize, conf=0.2)

    clss=[]
    Dictionary=results[0].names
    print(Dictionary)
    for i in range(len(results)):
        clss.append(results[i].boxes.cls.cpu().tolist())

    mapped_list = []
    # Iterate through each sublist in clss
    for sublist in clss:
        # Create a new sublist with mapped values
        mapped_sublist = [f'{int(num)}->' + Dictionary[int(num)] for num in sublist]
        # Append the mapped sublist to the mapped list
        mapped_list.append(mapped_sublist)

prepare_model(2000)

#objects: ->Dictionary
#0: 'BattleOfSeaChangeSize',
    #1: 'BattleOfSeaExit',
        #2: 'BattleOfSeaWindow',
            #3: 'CenterMyBoat',
                #4: 'ConfirmToBotQuestion',
                    #5: 'EnemyClicked',
                        #6: 'MyShip',
                            #7: 'TextFieldToBotQuestion',
                                #8: 'WarningBot',
                                    #9: 'WarningClose',
                                        #10: 'chat',
                                    #11: 'chatButton',
                                #12: 'fireDisabled',
                            #13: 'fireEnabled',
                        #14: 'glitter',
                    #15: 'glitterClicked',
                #16: 'map',
            #17: 'ocrText',
        #18: 'pointOfMoving'


WARNING ⚠️ imgsz=[2000] must be multiple of max stride 32, updating to [2016]
0: 1152x2016 1 BattleOfSeaChangeSize, 1 BattleOfSeaExit, 2 BattleOfSeaWindows, 1 CenterMyBoat, 1 ConfirmToBotQuestion, 1 TextFieldToBotQuestion, 1 chat, 1 chatButton, 1 fireDisabled, 1 ocrText, 701.5ms
Speed: 26.0ms preprocess, 701.5ms inference, 2.7ms postprocess per image at shape (1, 3, 1152, 2016)
{0: 'BattleOfSeaChangeSize', 1: 'BattleOfSeaExit', 2: 'BattleOfSeaWindow', 3: 'CenterMyBoat', 4: 'ConfirmToBotQuestion', 5: 'EnemyClicked', 6: 'MyShip', 7: 'TextFieldToBotQuestion', 8: 'WarningBot', 9: 'WarningClose', 10: 'chat', 11: 'chatButton', 12: 'fireDisabled', 13: 'fireEnabled', 14: 'glitter', 15: 'glitterClicked', 16: 'map', 17: 'ocrText', 18: 'pointOfMoving'}


We will create a function to delete the folder of results before from every use

In [7]:
import os
import shutil

def delete_folder(folder_path):
    """
    Check if the folder exists and delete it if it does.

    Args:
    - folder_path (str): The path to the folder to be checked and deleted.
    
    Returns:
    - bool: True if the folder was deleted successfully or didn't exist, False otherwise.
    """
    if os.path.exists(folder_path):
        try:
            shutil.rmtree(folder_path)
            print(f"The folder '{folder_path}' has been deleted.")
            return True
        except Exception as e:
            #print(f"Error deleting the folder '{folder_path}': {e}")
            return False
    else:
        #print(f"The folder '{folder_path}' does not exist.")
        return True  # Return True because the folder doesn't exist, no action required

# Example usage:
folder_path = r'C:\Users\Marios\Desktop\Μάριος\BotBattleOfSea\docs\src\runs'
delete_folder(folder_path)


The folder 'C:\Users\Marios\Desktop\Μάριος\BotBattleOfSea\docs\src\runs' has been deleted.


True

In [8]:
# Run batched inference on a list of images
model = YOLO(absolute_path_object_model_2000px_windows)

folder_path_to_read = r'C:\Users\Marios\Desktop\Μάριος\BotBattleOfSea\src\ObjectDetections'
delete_folder(folder_path)
results = model(folder_path_to_read, stream=True , conf=0.25, retina_masks=True , save=True, save_txt=True)  # return a generator of Results objects

# Process results generator
for result in results:
    boxes = result.boxes.cpu().numpy().data # Boxes object for bounding box outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    print(boxes)
    result.show()  # display to screen
    #result.save(filename='result.jpg')  # save to disk


[[     46.035      812.72      628.31        1017      0.9958          10]
 [     822.36      163.68      1100.4      193.19     0.99575           7]
 [     1014.4      123.74      1098.9      155.72     0.99033          17]
 [     1761.9      98.349      1888.5      224.08      0.9637          16]
 [     1844.3      4.8416      1854.1      14.605      0.9592           0]
 [     935.48      898.48      983.99      946.05     0.95685           3]
 [     913.88      230.93      1005.3      257.44     0.94305           4]
 [     673.88      952.37      718.05      998.43     0.91797          12]
 [     1724.6      98.401      1752.7      127.33     0.82556          11]
 [     76.213      8.4957      990.34      919.15     0.77689           2]
 [     1890.1      4.7806        1900      14.695     0.63226           1]
 [     716.28      37.551      1630.3      950.54     0.57607           2]
 [     1130.4      1030.9      1918.3        1080     0.29484           2]]
image 1/1 C:\Users\Mari

We will make this function available to use 

In [12]:
def predict(folder_path_to_read,model=YOLO(absolute_path_object_model_2000px_windows)):
    #folder_path_to_read = r'C:\Users\Marios\Desktop\Μάριος\BotBattleOfSea\src\ObjectDetections'
    delete_folder(folder_path)
    results = model(folder_path_to_read, stream=True , conf=0.25, retina_masks=True , save=True, save_txt=False )  # returns Results objects
    for result in results:
        boxes = result.boxes.cpu().numpy().data # Boxes object for bounding box outputs
        return boxes

def getCenterOfBoxes(ObjectArray):
    center_x = (ObjectArray[:, 0] + ObjectArray[:, 2]) / 2
    center_y = (ObjectArray[:, 1] + ObjectArray[:, 3]) / 2
    ObjectArray = np.column_stack((center_x, center_y, ObjectArray[:, 5])) # objects[:, 5] is the class of objects 
    return ObjectArray

#call the detection function
objects= predict(r'C:\Users\Marios\Desktop\Μάριος\BotBattleOfSea\src\ObjectDetections') #this is only for debugging purposes

final_array = getCenterOfBoxes(objects) #-> result: x,y,object_class 

final_array 

The folder 'C:\Users\Marios\Desktop\Μάριος\BotBattleOfSea\docs\src\runs' has been deleted.



array([[     337.17,      914.88,          10],
       [     961.37,      178.44,           7],
       [     1056.7,      139.73,          17],
       [     1825.2,      161.22,          16],
       [     1849.2,      9.7231,           0],
       [     959.74,      922.27,           3],
       [     959.57,      244.18,           4],
       [     695.96,       975.4,          12],
       [     1738.7,      112.86,          11],
       [     533.27,      463.82,           2],
       [     1895.1,      9.7379,           1],
       [     1173.3,      494.05,           2],
       [     1524.3,      1055.5,           2]], dtype=float32)

Now, lets create the logic of the bot. We want to find the nearest glitter->8 by the ship (6 -> 'MyShip')

In [13]:
row_of_myship = final_array[final_array[:, 2] == 6] #6 is the label of myShip
if(len(row_of_myship)!=0):
    x_ship = row_of_myship[0][0]
    y_ship = row_of_myship[0][1]
else:
    x_ship=float('inf')
    y_ship=float('inf')
# Calculate the Cartesian distance separately for each pair of points
distances = np.linalg.norm(np.column_stack((final_array[:,0], final_array[:,1])) - np.column_stack((x_ship, y_ship)), axis=1)
distances #0 means is the distance between myship to to myship

array([        inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf])

now lets create again the array with the labels and the distance between myship and the other objects inside of the function of getCenterOfBoxes!

In [14]:
def predict(folder_path_to_read,model=YOLO(absolute_path_object_model_2000px_windows)):
    #folder_path_to_read = r'C:\Users\Marios\Desktop\Μάριος\BotBattleOfSea\src\ObjectDetections'
    delete_folder(folder_path)
    results = model(folder_path_to_read, stream=True , conf=0.25, retina_masks=True , save=True, save_txt=False )  # returns Results objects
    for result in results:
        boxes = result.boxes.cpu().numpy().data # Boxes object for bounding box outputs
        return boxes

def getCenterOfBoxes(ObjectArray):
    center_x = (ObjectArray[:, 0] + ObjectArray[:, 2]) / 2
    center_y = (ObjectArray[:, 1] + ObjectArray[:, 3]) / 2
    ObjectArray = np.column_stack((center_x, center_y, ObjectArray[:, 5])) # objects[:, 5] is the class of objects 
    row_of_myship = ObjectArray[ObjectArray[:, 2] == 6] #6 is the label of myShip
    
    if(len(row_of_myship)!=0):
        x_ship = row_of_myship[0][0]
        y_ship = row_of_myship[0][1]
    else:
        x_ship=float('inf')
        y_ship=float('inf')

    distances = np.linalg.norm(np.column_stack((ObjectArray[:,0], ObjectArray[:,1])) - np.column_stack((x_ship, y_ship)), axis=1)
    ObjectArray = np.column_stack((center_x, center_y, distances, objects[:, 5]))

    return ObjectArray

#call the detection function
objects= predict(r'C:\Users\Marios\Desktop\Μάριος\BotBattleOfSea\src\ObjectDetections') #this is only for debugging purposes

final_array = getCenterOfBoxes(objects) 

final_array #-> result: x,y,distance,object_class 



The folder 'C:\Users\Marios\Desktop\Μάριος\BotBattleOfSea\docs\src\runs' has been deleted.



array([[     337.17,      914.88,         inf,          10],
       [     961.37,      178.44,         inf,           7],
       [     1056.7,      139.73,         inf,          17],
       [     1825.2,      161.22,         inf,          16],
       [     1849.2,      9.7231,         inf,           0],
       [     959.74,      922.27,         inf,           3],
       [     959.57,      244.18,         inf,           4],
       [     695.96,       975.4,         inf,          12],
       [     1738.7,      112.86,         inf,          11],
       [     533.27,      463.82,         inf,           2],
       [     1895.1,      9.7379,         inf,           1],
       [     1173.3,      494.05,         inf,           2],
       [     1524.3,      1055.5,         inf,           2]])

now lets get the coordinations of nearest glitter (#14) or some random glitter if the ship doeasn't appear/exist at screen

In [15]:
def predict(folder_path_to_read,model=YOLO(absolute_path_object_model_2000px_windows)):
    #folder_path_to_read = r'C:\Users\Marios\Desktop\Μάριος\BotBattleOfSea\src\ObjectDetections'
    delete_folder(folder_path)
    results = model(folder_path_to_read, stream=True , conf=0.25, retina_masks=True , save=True, save_txt=False )  # returns Results objects
    for result in results:
        boxes = result.boxes.cpu().numpy().data # Boxes object for bounding box outputs
        return boxes

def getCenterOfBoxes(ObjectArray):
    center_x = (ObjectArray[:, 0] + ObjectArray[:, 2]) / 2
    center_y = (ObjectArray[:, 1] + ObjectArray[:, 3]) / 2
    ObjectArray = np.column_stack((center_x, center_y, ObjectArray[:, 5])) # objects[:, 5] is the class of objects 
    row_of_myship = ObjectArray[ObjectArray[:, 2] == 6] #6 is the label of myShip
    
    if(len(row_of_myship)!=0):
        x_ship = row_of_myship[0][0]
        y_ship = row_of_myship[0][1]
    else:
        x_ship=float('inf')
        y_ship=float('inf')

    distances = np.linalg.norm(np.column_stack((ObjectArray[:,0], ObjectArray[:,1])) - np.column_stack((x_ship, y_ship)), axis=1)
    ObjectArray = np.column_stack((center_x, center_y, distances, objects[:, 5]))

    return ObjectArray


#sort array function:
def sort_array_by_column(array, column_index):
    """
    Sorts the given array based on the values in the specified column.

    Parameters:
    array (numpy.ndarray): The input array to be sorted.
    column_index (int): The index of the column based on which the sorting should be performed.

    Returns:
    numpy.ndarray: The sorted array.
    """
    # Check if the column index is valid
    if column_index < 0 or column_index >= array.shape[1]:
        raise ValueError("Column index is out of bounds.")

    # Sort the array by the values in the specified column
    sorted_indices = np.argsort(array[:, column_index])
    sorted_array = array[sorted_indices]

    return sorted_array

def nearest_glitter_or_random_glitter(final_array_sorted):
    label_14_glitters = final_array_sorted[final_array_sorted[:, 3] == 14] #14 is the label of glitter


    # Check if there are any glitters with label 2
    if len(label_14_glitters) == 0:
        return None
    x=label_14_glitters[0][0]
    y=label_14_glitters[0][1]
    
    return x,y
    
 
objects= predict(r'C:\Users\Marios\Desktop\Μάριος\BotBattleOfSea\src\ObjectDetections') #this is only for debugging purposes

final_array = getCenterOfBoxes(objects) 

sorted_array = sort_array_by_column(final_array,2) #2nd column is the column of distances
print(sorted_array)
coordinates = nearest_glitter_or_random_glitter(sorted_array)

pyautogui.moveTo(coordinates)
print(coordinates) #None if there is no Ship / x and y if exist both / x and y if only glitter exist

The folder 'C:\Users\Marios\Desktop\Μάριος\BotBattleOfSea\docs\src\runs' has been deleted.

[[     337.17      914.88         inf          10]
 [     961.37      178.44         inf           7]
 [     1056.7      139.73         inf          17]
 [     1825.2      161.22         inf          16]
 [     1849.2      9.7231         inf           0]
 [     959.74      922.27         inf           3]
 [     959.57      244.18         inf           4]
 [     695.96       975.4         inf          12]
 [     1738.7      112.86         inf          11]
 [     533.27      463.82         inf           2]
 [     1895.1      9.7379         inf           1]
 [     1173.3      494.05         inf           2]
 [     1524.3      1055.5         inf           2]]
None


Let's make the algorythm when a glitter_clicked exists to return None until the Ship goes to the glitter clicked

In [55]:
def predict(folder_path_to_read,model=YOLO(absolute_path_object_model_2000px_windows)):
    #folder_path_to_read = r'C:\Users\Marios\Desktop\Μάριος\BotBattleOfSea\src\ObjectDetections'
    delete_folder(folder_path)
    results = model(folder_path_to_read, stream=True , conf=0.15, retina_masks=True , save=True, save_txt=False )  # returns Results objects #!!must conf=0.15
    for result in results:
        boxes = result.boxes.cpu().numpy().data # Boxes object for bounding box outputs
        return boxes

def getCenterOfBoxes(ObjectArray):
    center_x = (ObjectArray[:, 0] + ObjectArray[:, 2]) / 2
    center_y = (ObjectArray[:, 1] + ObjectArray[:, 3]) / 2
    ObjectArray = np.column_stack((center_x, center_y, ObjectArray[:, 5])) # objects[:, 5] is the class of objects 
    row_of_myship = ObjectArray[ObjectArray[:, 2] == 6] #6 is the label of myShip
    
    if(len(row_of_myship)!=0):
        x_ship = row_of_myship[0][0]
        y_ship = row_of_myship[0][1]
    else:
        x_ship=float('inf')
        y_ship=float('inf')

    distances = np.linalg.norm(np.column_stack((ObjectArray[:,0], ObjectArray[:,1])) - np.column_stack((x_ship, y_ship)), axis=1)
    ObjectArray = np.column_stack((center_x, center_y, distances, objects[:, 5]))

    return ObjectArray


#sort array function:
def sort_array_by_column(array, column_index):
    """
    Sorts the given array based on the values in the specified column.

    Parameters:
    array (numpy.ndarray): The input array to be sorted.
    column_index (int): The index of the column based on which the sorting should be performed.

    Returns:
    numpy.ndarray: The sorted array.
    """
    # Check if the column index is valid
    if column_index < 0 or column_index >= array.shape[1]:
        raise ValueError("Column index is out of bounds.")

    # Sort the array by the values in the specified column
    sorted_indices = np.argsort(array[:, column_index])
    sorted_array = array[sorted_indices]

    return sorted_array

def nearest_glitter_or_random_glitter(final_array_sorted):
    label_14_glitters = final_array_sorted[final_array_sorted[:, 3] == 14] #14 -> glitter
    label_15_glitter_clicked = final_array_sorted[final_array_sorted[:, 3] == 15] #15 -> the label of glitter clicked
    label_03_stableCam = final_array_sorted[final_array_sorted[:, 3] == 3] #3: -> the label of CenterMyBoat 
    label_18_pointOfMoving = final_array_sorted[final_array_sorted[:, 3] == 18] #18 -> pointOfMoving 
    label_06_myShip = final_array_sorted[final_array_sorted[:, 3] == 6] #6 -> MyShip
    
    isGlitter = len(label_14_glitters)>0
    isGlitterClicked = len(label_15_glitter_clicked)>0
    isCameraMoving = len(label_03_stableCam)==0
    isBoatMoving = len(label_18_pointOfMoving)>0
    isMyShipOnScreen = len(label_06_myShip)>0
    
    print(f"case1:({isGlitter} or {isGlitterClicked}) and {isCameraMoving}")
    print(f"case2:({not isGlitter} and {not isCameraMoving})")

    if(isGlitter or isGlitterClicked) and isCameraMoving: #this will stable the camera and retake screenshot
        stableCam()  
        return None
    elif(not isGlitter and not isCameraMoving):
        centerCamera()



    if(len(label_15_glitter_clicked)!=0):
        time.sleep(3) #this is the time of waiting until the clicked glitter dissappears
        return None
    # Check if there are any glitters with label 2
    if isGlitter:
        x=label_14_glitters[0][0]
        y=label_14_glitters[0][1]
        return 'glitterToClick',x,y
    else:
        return 'navigation',None,None
    #navigation--------
    

    return None
 
objects= predict(r'C:\Users\Marios\Desktop\Μάριος\BotBattleOfSea\src\ObjectDetections') #this is only for debugging purposes


ocr_counter=1
final_array = getCenterOfBoxes(objects)
print(final_array)
if(handleOCRtext(objects, final_array, ocr_counter)):
    ocr_counter=ocr_counter+1

sorted_array = sort_array_by_column(final_array,2) #2nd column is the column of distances

order = nearest_glitter_or_random_glitter(sorted_array)
if(order is not None and order[0]=='glitterToClick'):
    pyautogui.moveTo((coordinates[1],coordinates[2]))
    print(coordinates) #None if there is no Ship / x and y if exist both / x and y if only glitter exist

The folder 'C:\Users\Marios\Desktop\Μάριος\BotBattleOfSea\docs\src\runs' has been deleted.

[[     337.17      914.88         inf          10]
 [     961.37      178.44         inf           7]
 [     1056.7      139.73         inf          17]
 [     1825.2      161.22         inf          16]
 [     1849.2      9.7231         inf           0]
 [     959.74      922.27         inf           3]
 [     959.57      244.18         inf           4]
 [     695.96       975.4         inf          12]
 [     1738.7      112.86         inf          11]
 [     533.27      463.82         inf           2]
 [     1895.1      9.7379         inf           1]
 [     1173.3      494.05         inf           2]
 [     1524.3      1055.5         inf           2]
 [     362.69      1054.8         inf           2]]
function: screenshot_array

0: 288x640 (no detections), 77.0ms
Speed: 2.0ms preprocess, 77.0ms inference, 0.0ms postprocess per image at shape (1, 3, 288, 640)
function: getText
case1:(False or

CheckPoint#1 (OCR and glitterCollector)

In [56]:
run = True
pyautogui.FAILSAFE = False #to move the mouse in corner (0,0)

while run:
    img = ImageGrab.grab(bbox=(0,0, pyautogui.size()[0], pyautogui.size()[1]))
    img_np = np.array(img)
    screen_bgr = cv2.cvtColor(img_np, cv2.COLOR_BGR2RGB)

    objects= predict(screen_bgr)
   
    if objects.size!=0:
        ocr_counter=1
        final_array = getCenterOfBoxes(objects)
        #print(final_array)
        if(handleOCRtext(objects, final_array, ocr_counter)):
            ocr_counter=ocr_counter+1

        sorted_array = sort_array_by_column(final_array,2) #2nd column is the column of distances

        order = nearest_glitter_or_random_glitter(sorted_array)
        

        print(order) #None if there is no Ship /
        if(order is not None and order[0]=='glitterToClick'):
            pyautogui.click((order[1],order[2]))
            pyautogui.moveTo(0, 0, duration=0)
        elif(order is not None and order[0]=='navigation'):
            label_16_map = objects[objects[:, 5] == 16] # 16 -> map
            if len(label_16_map) > 0:
                pyautogui.click(label_16_map[0][0]+5,label_16_map[0][1]+5)
                #κανοντας κλικ πανω αριστερα στο mini-χαρτη 
                #κανοντας κλικ μεσα στο χαρτη μεχρι να εμφανιστει το #18: 'pointOfMoving'      
                #Εκπαιδευση ζουμ, νησια και ψαρικά
        
        #objects: ->Dictionary
        #0: 'BattleOfSeaChangeSize',
        #1: 'BattleOfSeaExit',
        #2: 'BattleOfSeaWindow',
        #3: 'CenterMyBoat',
        #4: 'ConfirmToBotQuestion',
        #5: 'EnemyClicked',
        #6: 'MyShip',
        #7: 'TextFieldToBotQuestion',
        #8: 'WarningBot',
        #9: 'WarningClose',
        #10: 'chat',
        #11: 'chatButton',
        #12: 'fireDisabled',
        #13: 'fireEnabled',
        #14: 'glitter',
        #15: 'glitterClicked',
        #16: 'map',
        #17: 'ocrText',
        #18: 'pointOfMoving'       


    if keyboard.is_pressed('esc'):
            print("Stop Loop!")
            run = False

The folder 'C:\Users\Marios\Desktop\Μάριος\BotBattleOfSea\docs\src\runs' has been deleted.

The folder 'C:\Users\Marios\Desktop\Μάριος\BotBattleOfSea\docs\src\runs' has been deleted.

The folder 'C:\Users\Marios\Desktop\Μάριος\BotBattleOfSea\docs\src\runs' has been deleted.

case1:(False or False) and False
case2:(True and True) and True
function: centerCamera
('navigation', None, None)
The folder 'C:\Users\Marios\Desktop\Μάριος\BotBattleOfSea\docs\src\runs' has been deleted.

case1:(True or False) and True
case2:(False and False) and True
function: stableCam
None
The folder 'C:\Users\Marios\Desktop\Μάριος\BotBattleOfSea\docs\src\runs' has been deleted.

case1:(True or False) and False
case2:(False and True) and True
('glitterToClick', 1118.0284, 211.09497)
The folder 'C:\Users\Marios\Desktop\Μάριος\BotBattleOfSea\docs\src\runs' has been deleted.

case1:(True or False) and False
case2:(False and True) and True
('glitterToClick', 1117.9479, 211.29007)
The folder 'C:\Users\Marios\Desktop